<a href="https://colab.research.google.com/github/sizhky/AutoTrain/blob/master/nbs/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Data and Libraries

In [1]:
%%capture
%load_ext autoreload
%autoreload 2
# Setup imports
try:
    import timm
except:
    !pip install -U torch-snippets typer imgaug timm fastai

In [2]:
# Setup Data
from torch_snippets import *
from fastai.vision.all import *

# If running locally from AutoTrain/nbs
if not os.path.exists('/content') and P().resolve().stem != 'AutoTrain':
    %cd ..

# If running from colab
if os.path.exists('/content') and not os.path.exists('/content/AutoTrain'):
    !git clone https://github.com/sizhky/AutoTrain

# change present working directory
if not P().resolve().stem == 'AutoTrain':
    %cd AutoTrain
    
sys.path.append(str(P().resolve()))

/mnt/sda1/code/AutoTrain


## Verify Config

In [3]:
!cat configs/classification_imagenette.ini

[project]
owner: sizhky
name: imagenette
location: ${project.name}
model_directory: ${project.location}/models
type: classification
num_classes: 10
data_source_url: https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz

[architecture]
image_size: 224
[architecture.backbone]
model: efficientnet_b0
# everytime you change the model, vector size 
# must match the model's backbone output.
vector_size: 1280
[architecture.head]
@head: custom_head
head_input_size: ${architecture.backbone.vector_size}
num_classes: ${project.num_classes}

[training]
dir: /home/yyr/data/imagenette2-160
train_ratio: 0.8
[training.preprocess]
@preprocess_function: my_preprocess
image_size: ${architecture.image_size}

[training.data]
train_dir: ${training.dir}/train
validation_dir: ${training.dir}/validation

[training.scheme]
epochs: 2
freeze_epochs: 2
# initial_checkpoint: ${online_training.save_dir}
# mixup augmentation
mixup: 0.5
# cutmix
cutmix: 0.5 
mixup_switch_prob: 0.3
amp: true
# random augmentati

In [5]:
import os; os.environ['CONFIG'] = 'configs/classification_imagenette.ini'
!python main.py train --task=classification

Traceback (most recent call last):
  File "/mnt/sda1/code/AutoTrain/main.py", line 34, in <module>
    cli()
  File "/home/yyr/miniconda3/lib/python3.9/site-packages/typer/main.py", line 214, in __call__
    return get_command(self)(*args, **kwargs)
  File "/home/yyr/miniconda3/lib/python3.9/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/home/yyr/miniconda3/lib/python3.9/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/home/yyr/miniconda3/lib/python3.9/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/yyr/miniconda3/lib/python3.9/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/yyr/miniconda3/lib/python3.9/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/home/yyr/miniconda3/lib/python3.9/site-packages/typer/main.py"

In [5]:
import os; os.environ['CONFIG'] = 'configs/classification_imagenette.ini'
!python main.py infer --task=classification --folder='imagenette2-160/val/n02102040/'

2021-09-28 23:14:24.824 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'
2021-09-28 23:14:31.105 | INFO     | auto_train.classification.infer:<module>:6 - Loaded weights from imagenette/models/trained-model-v1/model.pth to given model
2021-09-28 23:14:31.644 | DEBUG    | __main__:infer:31 - 
[
    {
        "file": "/mnt/sda1/code/AutoTrain/imagenette2-160/val/n02102040/n02102040_6892.JPEG",
        "prediction": "n02102040",
        "confidence": "50.11%"
    },
    {
        "file": "/mnt/sda1/code/AutoTrain/imagenette2-160/val/n02102040/n02102040_3640.JPEG",
        "prediction": "n02102040",
        "confidence": "67.61%"
    },
    {
        "file": "/mnt/sda1/code/AutoTrain/imagenette2-160/val/n02102040/n02102040_1980.JPEG",
        "prediction": "n02102040",
        "confidence": "98.55%"
    },
    {
        "file": "/mnt/sda1/code/AutoTrain/imagenette2-160/val/n02102040/n02102